In [43]:
#### assigining values from the saved files ###########

In [ ]:
import json
with open("route_bus_detail_dict.txt", 'r') as file1:
    route_bus_detail_dict = json.load(file1)
with open("route_travel_date_dict.txt", 'r') as file2:
    route_travel_date_dict = json.load(file2)

In [ ]:
####################################### Pre-processing the data before inserting data into DB ###########################################

In [51]:
route_bus_prep = []
for r,i in route_bus_detail_dict.items():

    for j in i:
        tem = {}
        sr_present = 0
        for k,l in j.items():
            # primary key(id) ----- to be added in DB and it should be series type (self_incremental)
            
            tem['Bus Route Name'] = r
            tem['Bus Routes Link'] = route_travel_date_dict[r]['link']
            tem['Travel Date'] = route_travel_date_dict[r]['travel_date']

            if 'travels lh-24' in k:
                tem['Bus Name'] = l
            
            elif 'bus-type' in k:
                tem['Bus Type'] = l
            
            elif 'dp-time' in k:
                tem['Departing Time'] = l         #### change it as time --- data type
            
            elif 'dur l-color' in k:
                tem['Duration'] = l

            elif 'bp-time' in k:
                tem['Reaching Time'] = l         #### change it as time --- data type
                
            elif 'rating-sec' in k:
                tem['Star Rating'] = l
                
            elif 'rating' in k:
                tem['Star Rating'] = l
                    
            elif 'fare d-block' in k:
                tem['Price'] = l
                
            elif 'seat-left' in k:
                tem['Seat Availability'] = int(l.split()[0])  ##### use split and take the 1st value alone
    
        
        route_bus_prep.append(tem)


In [52]:
import pandas as pd

In [53]:
df_bus = pd.DataFrame(route_bus_prep)


In [54]:
df_bus_backup = df_bus.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12271 entries, 0 to 12270
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Bus Route Name     12271 non-null  object
 1   Bus Routes Link    12271 non-null  object
 2   Travel Date        12271 non-null  object
 3   Bus Name           12271 non-null  object
 4   Bus Type           12271 non-null  object
 5   Departing Time     12271 non-null  object
 6   Duration           12271 non-null  object
 7   Reaching Time      12271 non-null  object
 8   Star Rating        12213 non-null  object
 9   Price              12271 non-null  object
 10  Seat Availability  12271 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1.0+ MB


In [57]:
from datetime import datetime, timedelta

In [58]:
df_bus["Travel Date"] = pd.to_datetime(df_bus["Travel Date"] + " 2024", format="%d %b %Y")

In [59]:
df_bus["Departing Time"] = pd.to_timedelta(df_bus["Departing Time"].str.extract(r"(\d{2}:\d{2})")[0] + ":00")

In [60]:
# df_bus["Duration"] = df_bus["Duration"].apply(lambda x: timedelta(hours=int(x.split("h")[0]), minutes=int(x.split("h")[1].split("m")[0])))
def parse_duration(duration):
    try:
        hours, minutes = map(int, duration.replace("h", "").replace("m", "").split())
        return timedelta(hours=hours, minutes=minutes)
    except Exception:
        # Handle invalid values
        return timedelta(0) 

In [61]:
df_bus["Duration"] = df_bus["Duration"].apply(parse_duration)

In [62]:
df_bus["Reaching Date"] = df_bus["Travel Date"] + df_bus["Departing Time"] + df_bus["Duration"]

In [63]:
df_bus["Starting Date"] = df_bus["Travel Date"] + df_bus["Departing Time"]

In [75]:
df_bus['Star Rating'] = df_bus['Star Rating'].fillna('0')
df_bus['Star Rating'] = df_bus['Star Rating'].str.replace("New", '0')
df_bus['Star Rating'] = df_bus['Star Rating'].astype(float)
df_bus['Star Rating'] = df_bus['Star Rating'].apply(lambda x : '0' if x == None else x)

In [65]:
df_bus["Price"] = df_bus["Price"].str.replace("INR ", "").astype(float)


In [66]:
df_bus_final = df_bus[['Bus Route Name', 'Bus Routes Link','Bus Name','Bus Type', 'Starting Date','Duration','Reaching Date', 'Star Rating', 'Price', 'Seat Availability']]

In [67]:
df_bus_final["Duration"] = df_bus_backup["Duration"].copy()
df_bus_final["Duration"] = df_bus_final["Duration"].astype(str)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14992\3698342266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bus_final["Duration"] = df_bus_backup["Duration"].copy()
C:\Users\Hp\AppData\Local\Temp\ipykernel_14992\3698342266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bus_final["Duration"] = df_bus_final["Duration"].astype(str)


In [ ]:
#### inserting data into a table in Database ### postgesql

In [22]:
#!pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
import psycopg2

In [89]:
from psycopg2.extras import execute_values

In [90]:
conn = psycopg2.connect(
    database="Redbus",
    user="postgres",
    password="12345678",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

In [94]:
cur.execute(""" CREATE TABLE bus_routes (
id SERIAL PRIMARY KEY,
route_name TEXT not null,
route_link TEXT,
busname TEXT not null,
bustype TEXT,
departing_time TIME,
duration TEXT,
reaching_time TIME,
star_rating FLOAT,
price DECIMAL(10,2),
seats_available INT
)""")

#### data type --- DATETIME is not available in postgresql


UndefinedObject: type "datetime" does not exist
LINE 7: departing_time DATETIME,
                       ^


In [143]:
conn.commit()

In [78]:
# Rename DataFrame columns to match the database table
df_bus_final.rename(columns={
    'Bus Route Name': 'route_name',
    'Bus Routes Link': 'route_link',
    'Bus Name': 'busname',
    'Bus Type': 'bustype',
    'Starting Date': 'departing_time',
    'Duration': 'duration',
    'Reaching Date': 'reaching_time',
    'Star Rating': 'star_rating',
    'Price': 'price',
    'Seat Availability': 'seats_available'
}, inplace=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14992\814891537.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bus_final.rename(columns={


In [91]:
    columns = ', '.join(df_bus_final.columns)
    table_name = 'bus_routes'
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES %s"

    # Convert DataFrame rows to a list of tuples
    values = [tuple(row) for row in df_bus_final.to_numpy()]

    # Use execute_values for efficient bulk inserts
    execute_values(cur, insert_query, values)

In [93]:
conn.commit()